In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

# Load election data
data_dir = "../data/"
income = pd.read_csv(data_dir + "income_by_municipality.csv", sep=',')
income

In [ ]:
# Convert the 'Total' column to numeric using .loc
income.loc[:, 'Total'] = income['Total'].apply(pd.to_numeric, errors='coerce')
income

In [ ]:
income_municipality = income.groupby('Municipalities')['Total'].mean().reset_index()
income_municipality

In [ ]:
# Load GeoJSON file
map_catalonia='municipality_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)

In [ ]:
# Translate column used to merge datasets and sort dataset based on this
geo_data.rename(columns={'nom_muni': 'Municipalities'}, inplace=True)
geo_data=geo_data.sort_values(by='Municipalities')

In [ ]:
# Merge municipalities map and income by municipality
merged_data = geo_data.merge(income_municipality, on='Municipalities')
merged_data = merged_data.rename(columns={'Total': 'Average income per household (€)'})
merged_data

In [ ]:
# Plot the average income on a map of Catalonian municipalities

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Average income per household (€)',  # Change to the column you want to visualize
    hover_name='Municipalities',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7,
    title='Average income by municipality'
)
fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [ ]:
# Assuming you have loaded both datasets into dataframes
munic_county_prov_inh = pd.read_csv(data_dir + "municipality_county_province_cat.csv")

# Merge the two datasets on the 'Municipalities' column
result_df = pd.merge(income_municipality, munic_county_prov_inh[['Municipality_name', 'County_name', 'Province_name']], left_on='Municipalities', right_on='Municipality_name', how='left')

# Drop the duplicate 'Municipality_name' column
result_df = result_df.drop(columns='Municipality_name')

# Display the result
print(result_df)


In [ ]:
# Group by 'County_name' and calculate the average 'Total'
average_income_by_county = result_df.groupby('County_name')['Total'].mean().reset_index()

# Display the result
print(average_income_by_county)


In [ ]:
# Load GeoJSON file
map_catalonia='county_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)
# Rename column of interest and sort data by the latter
geo_data.rename(columns={'nomcomar': 'County_name'}, inplace=True)
geo_data_sorted=geo_data.sort_values(by='County_name')

In [ ]:
# Merge county map and votes by county
merged_data = geo_data_sorted.merge(average_income_by_county, on='County_name')
merged_data = merged_data.rename(columns={'Total': 'Average income per household (€)'})

merged_data

In [ ]:
# Plot the election results on a map of Catalonian counties

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Average income per household (€)',  # Change to the column you want to visualize
    hover_name='County_name',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7,
    color_discrete_sequence=['red', 'gold','seagreen'],
    title='Average income per household by county'
)

fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [ ]:
# Load the province map
map_catalonia='province_map.geojson'
geo_data = gpd.read_file(data_dir + map_catalonia)
# Rename column of interest and sort data by the latter
geo_data.rename(columns={'nomprov': 'Province_name'}, inplace=True)
geo_data_sorted=geo_data.sort_values(by='Province_name')


In [ ]:
# Group by 'Province_name' and calculate the average 'Total'
average_income_by_province = result_df.groupby('Province_name')['Total'].mean().reset_index()

# Display the result
print(average_income_by_province)

In [ ]:
# Merge province map and votes by province
merged_data = geo_data_sorted.merge(average_income_by_province, on='Province_name')
merged_data = merged_data.rename(columns={'Total': 'Average income per household (€)'})

merged_data

In [ ]:
# Plot the election results on a map of Catalonian provinces
#import kaleido
fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Average income per household (€)',  # Change to the column you want to visualize
    hover_name='Province_name',
    mapbox_style="carto-positron",
    center={"lat": 41.8781, "lon": 1.7834},  # Center of Catalonia
    zoom=7,
    color_discrete_sequence=['red', 'gold'],
    title='Average income per household by province'
)
fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()